<a href="https://colab.research.google.com/github/sasichon07/HealthCheck-/blob/main/HealthCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def calculate_bmi(weight, height):
    bmi = weight / (height / 100) ** 2
    if bmi < 18.5:
        result = "น้ำหนักต่ำกว่าเกณฑ์"
    elif 18.5 <= bmi < 23:
        result = "น้ำหนักปกติ"
    elif 23 <= bmi < 24.9:
        result = "น้ำหนักเกิน"
    elif 25 <= bmi < 29.9:
        result = "โรคอ้วนระดับที่ 1"
    else:
        result = "โรคอ้วนระดับที่ 2"
    return bmi, result

def load_food_data():
    df = pd.read_excel("ปริมาณพลังงาน.xlsx")
    return dict(zip(df['ชื่อเมนูอาหาร'], df['ปริมาณแคลอรี่']))

def load_ingredient_data():
    df = pd.read_excel("แคลในวัตถุดิบ.xlsx")
    return dict(zip(df['Food'], df['Calories']))

def get_food_energy(food_list, food_data):
    total_energy = sum(float(food_data.get(food, 0)) for food in food_list if food)
    return total_energy

def get_ingredient_energy(ingredient_data):
    total_energy = 0
    while True:
        ingredient = input("กรอกชื่อวัตถุดิบ (พิมพ์ - เพื่อหยุด): ").strip()
        if ingredient == "-":
            break
        if ingredient in ingredient_data:
            try:
                amount = float(input(f"กรอกปริมาณ {ingredient} (กรัม): "))
                energy = (ingredient_data[ingredient] / 100) * amount
                total_energy += energy
                print(f"{ingredient}: {energy:.2f} kcal\n")
            except ValueError:
                print("กรุณากรอกตัวเลขที่ถูกต้อง")
        else:
            print("❌ วัตถุดิบนี้ไม่มีในฐานข้อมูล")

    return total_energy

def calculate_energy_needs(sex, age, activity_level):
    energy_needs = {
        "ชาย": {"ต่ำ": {2: 1000, 4: 1400, 9: 1800, 14: 2200, 19: 2400, 31: 2200, 51: 2000},
                "ปานกลาง": {2: 1400, 4: 1600, 9: 2200, 14: 2800, 19: 2800, 31: 2600, 51: 2400},
                "สูง": {2: 1400, 4: 2000, 9: 2600, 14: 3200, 19: 3000, 31: 3000, 51: 2800}},
        "หญิง": {"ต่ำ": {2: 1000, 4: 1200, 9: 1600, 14: 1800, 19: 2000, 31: 1800, 51: 1600},
                "ปานกลาง": {2: 1400, 4: 1600, 9: 2000, 14: 2000, 19: 2200, 31: 2000, 51: 1800},
                "สูง": {2: 1400, 4: 1800, 9: 2200, 14: 2400, 19: 2400, 31: 2200, 51: 2000}}
    }
    for age_range in sorted(energy_needs[sex][activity_level].keys()):
        if age < age_range:
            return energy_needs[sex][activity_level][age_range]
    return energy_needs[sex][activity_level][51]

def health_advice(bmi):
    if bmi < 18.5:
        return "ควรเพิ่มอาหารที่มีโปรตีนและพลังงานมากขึ้น"
    elif 18.5 <= bmi <= 22.9:
        return "น้ำหนักปกติ ควรรักษาสุขภาพให้ดี"
    elif 23 <= bmi < 25:
        return "ควรควบคุมอาหารและออกกำลังกาย"
    elif 25 <= bmi < 30:
        return "ควรลดปริมาณไขมันและออกกำลังกายมากขึ้น"
    else:
        return "ควรลดอาหารที่มีแคลอรี่สูงและออกกำลังกายจริงจัง"

def food_calculator(food_data, ingredient_data, sex, age, activity_level):
    recommended_energy = calculate_energy_needs(sex, age, activity_level)
    energy_per_meal = recommended_energy / 3

    while True:
        print("\nเลือกโหมดการคำนวณ:")
        print("1: คำนวณพลังงานที่ได้รับใน 1 วัน (3 มื้อ)")
        print("2: คำนวณแคลอรี่จากวัตถุดิบที่ใช้ทำอาหาร")
        mode = input("กรุณาเลือก (1 หรือ 2): ")

        if mode == "1":
            food_morning = input("มื้อเช้า: ")
            food_lunch = input("มื้อกลางวัน: ")
            food_dinner = input("มื้อเย็น: ")
            daily_intake = get_food_energy([food_morning, food_lunch, food_dinner], food_data)
            balance = daily_intake - recommended_energy

            print("\n--- ผลลัพธ์ ---")
            print(f"รวมพลังงานจากอาหาร: {daily_intake:.2f} kcal")
            print(f"พลังงานที่แนะนำต่อวัน: {recommended_energy} kcal")
            print(f"พลังงานที่ขาดหรือเกิน: {balance:.2f} kcal")
            print("✅ คุณกินอยู่ในช่วงที่เหมาะสม" if balance == 0 else "⚠️ ควรปรับสมดุลพลังงาน")

        elif mode == "2":
            total_energy = get_ingredient_energy(ingredient_data)
            balance = total_energy - energy_per_meal

            print("\n--- ผลลัพธ์ ---")
            print(f"พลังงานจากวัตถุดิบ: {total_energy:.2f} kcal")
            print(f"พลังงานที่แนะนำต่อมื้อ: {energy_per_meal:.2f} kcal")
            print(f"พลังงานที่ขาดหรือเกินจากที่แนะนำ: {balance:.2f} kcal")
            print("✅ พลังงานเหมาะสม" if -100 <= balance <= 100 else "⚠️ อาจต้องปรับปริมาณอาหาร")

        else:
            print("❌ ตัวเลือกไม่ถูกต้อง กรุณาเลือกใหม่")

        choice = input("\nต้องการทำงานต่อหรือไม่? (1 = ต่อ, 2 = หยุด): ")
        if choice == "2":
            print("\nขอบคุณที่ใช้บริการ ขอให้มีสุขภาพดี 😊")
            break

def main():
    food_data = load_food_data()
    ingredient_data = load_ingredient_data()

    print("🎯 ยินดีต้อนรับเข้าสู่โปรแกรมสุขภาพ")
    sex = input("เพศ (ชาย/หญิง): ")
    age = int(input("อายุ (ปี): "))
    weight = float(input("น้ำหนัก (กก.): "))
    height = float(input("ส่วนสูง (ซม.): "))
    activity_level = input("ระดับการออกกำลังกาย (ต่ำ/ปานกลาง/สูง): ")

    bmi, bmi_result = calculate_bmi(weight, height)
    print(f"\n📊 BMI ของคุณ: {bmi:.2f} ({bmi_result})")
    print(f"💡 คำแนะนำ: {health_advice(bmi)}")
    recommended_energy = calculate_energy_needs(sex, age, activity_level)
    print(f"🔥 พลังงานที่ควรได้รับใน 1 วัน: {recommended_energy} kcal")

    food_calculator(food_data, ingredient_data, sex, age, activity_level)

if __name__ == "__main__":
    main()
